# Amazon SageMaker Model - Model Runner with Holdout Data

## Section 1 - Setup <a id='setup'></a>

#### 1.1 Import necessary libraries

In [1]:
%%time

from datetime import datetime, timedelta, timezone
import json
import os
import re
import boto3
from time import sleep
from threading import Thread

import io
import random
import pandas as pd

from sagemaker import get_execution_role, session, Session, image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.processing import ProcessingJob
from sagemaker.serializers import CSVSerializer

from sagemaker.model import Model
from sagemaker.model_monitor import (
    BiasAnalysisConfig,
    CronExpressionGenerator,
    DataCaptureConfig,
    EndpointInput,
    ExplainabilityAnalysisConfig,
    ModelBiasMonitor,
    ModelExplainabilityMonitor,
)

from sagemaker.clarify import (
    BiasConfig,
    DataConfig,
    ModelConfig,
    ModelPredictedLabelConfig,
    SHAPConfig,
)

from threading import Timer

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
CPU times: user 1.22 s, sys: 139 ms, total: 1.36 s
Wall time: 1.56 s


#### 1.2 AWS region and IAM Role & Sagemaker Clients and Bucket Details

In [2]:
sagemaker_session = Session()
sagemaker_client = sagemaker_session.sagemaker_client
sagemaker_runtime_client = sagemaker_session.sagemaker_runtime_client

# Retrieve the default Amazon S3 bucket associated with the SageMaker session.
bucket = sagemaker_session.default_bucket()
print("Bucket:", bucket)

# Get the IAM role associated with the current SageMaker notebook or environment.
role = get_execution_role()
print("RoleArn:", role)

# Get the AWS region name for the current session.
region = boto3.Session().region_name
print("Region", region)

# Retrieve the AWS account ID of the caller using the Security Token Service (STS) client.
account_id = boto3.client("sts").get_caller_identity().get("Account")

# Create a Boto3 client for the SageMaker service, specifying the AWS region.
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

# Create an S3 client
s3 = boto3.client('s3')

Bucket: sagemaker-us-east-1-084375567266
RoleArn: arn:aws:iam::084375567266:role/service-role/SageMaker-ExecutionRole-20250222T162355
Region us-east-1


In [3]:
endpoint_name = "retain-ai-xgb-endpoint"
response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
display(response)

{'EndpointName': 'retain-ai-xgb-endpoint',
 'EndpointArn': 'arn:aws:sagemaker:us-east-1:084375567266:endpoint/retain-ai-xgb-endpoint',
 'EndpointConfigName': 'retain-ai-xgb-endpoint',
 'ProductionVariants': [{'VariantName': 'AllTraffic',
   'DeployedImages': [{'SpecifiedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:0.90-1-cpu-py3',
     'ResolvedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost@sha256:4814427c3e0a6cf99e637704da3ada04219ac7cd5727ff62284153761d36d7d3',
     'ResolutionTime': datetime.datetime(2025, 2, 24, 3, 30, 46, 86000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1}],
 'DataCaptureConfig': {'EnableCapture': True,
  'CaptureStatus': 'Started',
  'CurrentSamplingPercentage': 100,
  'DestinationS3Uri': 's3://sagemaker-us-east-1-084375567266/aai-540-group-3-final-project/data/monitoring'},
 'EndpointStatus': 'InService',
 'CreationTime': date

In [4]:
# Download the file from S3 to a local file object
houldout_file_key = "aai-540-group-3-final-project/data/holdout/holdout.csv"
response = s3.get_object(Bucket=bucket, Key=houldout_file_key)

# Read the content of the file into a pandas DataFrame
data_df = pd.read_csv(response['Body'])

# Display the DataFrame
display(data_df)

all_headers = data_df.columns.to_list()
print(all_headers)

label_header = all_headers[len(all_headers) - 1]
print(label_header)

,Employee ID,Age,Gender,Years at Company,Job Role,Monthly Income,Work-Life Balance,Job Satisfaction,Performance Rating,Number of Promotions,...,Number of Dependents,Job Level,Company Size,Company Tenure,Remote Work,Leadership Opportunities,Innovation Opportunities,Company Reputation,Employee Recognition,Attrition
0,44132,38,1,23,1,10351,0,2,1,2,...,4,0,2,54,0,0,0,3,2,1
1,67355,22,1,13,2,8012,1,0,2,0,...,1,2,1,74,0,0,0,2,0,1
2,67290,40,0,32,4,6157,1,1,1,0,...,4,1,2,62,0,0,0,3,1,0
3,25581,33,0,8,2,9281,2,0,0,2,...,0,0,1,25,0,0,0,2,1,0
4,39986,57,1,22,3,6116,3,0,0,2,...,2,1,1,38,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29794,37472,32,1,13,2,8809,1,2,0,2,...,4,1,1,26,0,0,0,2,2,1
29795,16814,45,0,16,4,9907,2,0,0,0,...,1,0,2,79,1,0,0,3,2,0
29796,15541,28,1,3,3,5238,0,0,3,0,...,1,1,1,5,0,0,0,2,0,0
29797,56431,39,1,2,1,4814,2,2,0,2,...,2,1,1,74,0,1,0,2,1,0


['Employee ID', 'Age', 'Gender', 'Years at Company', 'Job Role', 'Monthly Income', 'Work-Life Balance', 'Job Satisfaction', 'Performance Rating', 'Number of Promotions', 'Overtime', 'Distance from Home', 'Education Level', 'Marital Status', 'Number of Dependents', 'Job Level', 'Company Size', 'Company Tenure', 'Remote Work', 'Leadership Opportunities', 'Innovation Opportunities', 'Company Reputation', 'Employee Recognition', 'Attrition']
Attrition


#### 1.3 S3 bucket and prefixes

In [5]:
# Bucket prefix to store details for the monitor
prefix = "aai-540-group-3-final-project"

# Other prefixes
data_capture_prefix = f"{prefix}/data/monitoring"
s3_capture_upload_path = f"s3://{bucket}/{data_capture_prefix}"

ground_truth_upload_path = (
    f"s3://{bucket}/{prefix}/ground_truth_data/{datetime.now():%Y-%m-%d}"
)

reports_prefix = f"{prefix}/reports"
s3_report_path = f"s3://{bucket}/{reports_prefix}"

##Get the model monitor image
monitor_image_uri = image_uris.retrieve(framework="model-monitor", region=region)

print("Image URI:", monitor_image_uri)
print(f"Capture path: {s3_capture_upload_path}")
print(f"Ground truth path: {ground_truth_upload_path}")
print(f"Report path: {s3_report_path}")

Image URI: 156813124566.dkr.ecr.us-east-1.amazonaws.com/sagemaker-model-monitor-analyzer
Capture path: s3://sagemaker-us-east-1-084375567266/aai-540-group-3-final-project/data/monitoring
Ground truth path: s3://sagemaker-us-east-1-084375567266/aai-540-group-3-final-project/ground_truth_data/2025-02-25
Report path: s3://sagemaker-us-east-1-084375567266/aai-540-group-3-final-project/reports


## 2. Call Endpoint with Houldout Data & Upload Ground Truth

In [26]:
import matplotlib.pyplot as plt

def print_predictions_graphically(predictions):
    """
    Print predictions graphically as dots and dashes.

    Parameters:
    - predictions (list): List of predictions (either probabilities or class labels).
    """
    for prediction in predictions:
        try:
            # Convert prediction to float if it's not already
            pred_value = float(prediction)

            # Print a dot for values >= 0.5, and a dash for values < 0.5
            if pred_value >= 0.5:
                print(".", end="", flush=True)
            else:
                print("-", end="", flush=True)

        except ValueError:
            print("Invalid prediction value:", prediction)
    
    print()  # Move to the next line after all predictions are printed

In [44]:
# Function to invoke SageMaker endpoint and track predictions
def invoke_sagemaker_endpoint(data_df=data_df, num_rows=20, endpoint_name="retain-ai-xgb-endpoint"):
    """
    Prepare data and send it to a SageMaker endpoint for prediction.
    """
    # Select random rows
    rows = random.sample(range(len(data_df)), num_rows)
    selected_data = data_df.iloc[rows]
    
    # Drop the 'Attrition' column as it's the target variable
    input_data = selected_data.drop(columns=['Attrition'])
    
    # Create a SageMaker runtime client
    runtime_client = boto3.client('sagemaker-runtime')

    # Loop through the selected rows
    predictions = []
    employee_ids = []

    for index, row in input_data.iterrows():
        row_data = row.to_frame().T  # Convert the row to a DataFrame (single-row)
        
        # Convert to CSV format without the header
        csv_buffer = io.StringIO()
        row_data.to_csv(csv_buffer, index=False, header=False)
        row_payload = csv_buffer.getvalue()  # Get the CSV payload for the row
        
        # Invoke the endpoint for the row
        response = runtime_client.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType='text/csv',  # Assuming the model expects CSV input
            Body=row_payload,
            InferenceId=str(row['Employee ID'])
        )
        
        # Get the prediction from the response
        prediction = response['Body'].read().decode('utf-8')
        predictions.append(prediction)
        # Store the Employee ID for the prediction
        employee_ids.append(row['Employee ID'])
        
    return predictions, employee_ids

In [48]:
# Do some test predictions 
display(invoke_sagemaker_endpoint(data_df, 20))

(['0.8406565189361572',
  '0.3030962347984314',
  '0.4130893051624298',
  '0.8846337199211121',
  '0.9124721884727478',
  '0.1122896745800972',
  '0.818571925163269',
  '0.9331226944923401',
  '0.7598685622215271',
  '0.8436989784240723',
  '0.15304571390151978',
  '0.27725473046302795',
  '0.34751754999160767',
  '0.39823800325393677',
  '0.8239598274230957',
  '0.935815691947937',
  '0.9508293271064758',
  '0.7514158487319946',
  '0.6431967616081238',
  '0.830248236656189'],
 [66561,
  50726,
  43405,
  74243,
  2806,
  21954,
  43669,
  48977,
  53638,
  26519,
  15183,
  72354,
  53410,
  55247,
  1406,
  17578,
  22715,
  25489,
  8757,
  28279])

In [12]:
# !aws s3 ls $s3_capture_upload_path  --recursive

In [49]:
# View Capture data
print("Waiting for captures to show up", end="")
capture_files = sorted(S3Downloader.list(f"{s3_capture_upload_path}/{endpoint_name}"))
if capture_files:
    capture_file = S3Downloader.read_file(capture_files[-1]).split("\n")
    capture_record = json.loads(capture_file[0])
    display(capture_record)
print()
print("Found Capture Files:")
print("\n ".join(capture_files[-3:]))

Waiting for captures to show up

{'captureData': {'endpointInput': {'observedContentType': 'text/csv',
   'mode': 'INPUT',
   'data': '23435,57,1,14,3,5812,3,0,0,0,0,91,3,2,4,1,1,78,0,0,0,3,0\n',
   'encoding': 'CSV'},
  'endpointOutput': {'observedContentType': 'text/csv; charset=utf-8',
   'mode': 'OUTPUT',
   'data': '0.3607085049152374',
   'encoding': 'CSV'}},
 'eventMetadata': {'eventId': '8043caff-e40d-4f8b-9568-aa478ba1b9f6',
  'inferenceId': '23435',
  'inferenceTime': '2025-02-25T05:00:11Z'},
 'eventVersion': '0'}


Found Capture Files:
s3://sagemaker-us-east-1-084375567266/aai-540-group-3-final-project/data/monitoring/retain-ai-xgb-endpoint/AllTraffic/2025/02/25/04/34-25-689-a470d254-2652-4cf7-98cd-7793652fceb7.jsonl
 s3://sagemaker-us-east-1-084375567266/aai-540-group-3-final-project/data/monitoring/retain-ai-xgb-endpoint/AllTraffic/2025/02/25/04/35-27-225-a384fb7c-f9f2-4821-a1ff-731d43d081ec.jsonl
 s3://sagemaker-us-east-1-084375567266/aai-540-group-3-final-project/data/monitoring/retain-ai-xgb-endpoint/AllTraffic/2025/02/25/05/00-11-355-15ce83eb-3497-4fd8-84cd-868e76d534a4.jsonl


In [50]:
# To store predictions and their corresponding employee IDs
predictions_store = []
employee_ids_store = []

# Flag to stop the scheduler
stop_scheduler = False

# Function to invoke SageMaker endpoint and track predictions
def invoke_sagemaker_endpoint_with_schedule(data_df=data_df, num_rows=20, endpoint_name="retain-ai-xgb-endpoint"):
    if stop_scheduler:
        print("Prediction scheduler stopped.")
        return
        
    predictions, employee_ids = invoke_sagemaker_endpoint(data_df=data_df, num_rows=20, endpoint_name="retain-ai-xgb-endpoint")
    print_predictions_graphically(predictions)
    # Store predictions and IDs for later ground truth upload
    predictions_store.append(predictions)
    employee_ids_store.append(employee_ids)

    # Rescedule every 10 secs 
    Timer(10, invoke_sagemaker_endpoint_with_schedule).start()

In [51]:
# Function to upload ground truth every hour
def upload_ground_truth(upload_time=None):
    if stop_scheduler:
        print("Upload scheduler stopped.")
        return
        
    if upload_time is None:
        # Get current time for timestamping the folder in S3
        upload_time = datetime.utcnow()

    # Flatten the stored Employee IDs into a list of ground truth records
    ground_truth_records = []
    for ids in employee_ids_store:  # Only need employee_ids for actual labels
        for emp_id in ids:  # For each Employee ID
            # Fetch the actual Attrition label from data_df for the given Employee ID
            actual_attrition = data_df.loc[data_df['Employee ID'] == emp_id, 'Attrition'].values[0]

            # Now use the actual Attrition label (not the predicted one) for ground truth
            ground_truth_records.append({
                "groundTruthData": {
                    "data": str(actual_attrition),  # Use the actual Attrition value (1 or 0)
                    "encoding": "CSV",
                },
                "eventMetadata": {
                    "eventId": str(emp_id),  # Use Employee ID as the unique identifier
                },
                "eventVersion": "0",
            })

    # Convert ground truth records to JSONL (newline-delimited JSON)
    upload_records = [json.dumps(record) for record in ground_truth_records]
    data_to_upload = "\n".join(upload_records)
    
    # Upload to S3
    target_s3_uri = f"{ground_truth_upload_path}/{upload_time:%Y/%m/%d/%H/%M%S}.jsonl"
    print(f"Uploading {len(upload_records)} records to", target_s3_uri)
    S3Uploader.upload_string_as_file_body(data_to_upload, target_s3_uri)
   
    # Clear the stored Employee IDs after uploading
    employee_ids_store.clear()
   
    # Rescedule every 60 secs 
    Timer(60, upload_ground_truth).start()

In [52]:
# Start the prediction scheduler
def start_predictions():
    global stop_scheduler
    stop_scheduler = False  # Ensure that the scheduler is running
    invoke_sagemaker_endpoint_with_schedule()

In [53]:
def start_upload():
    global stop_scheduler
    stop_scheduler = False  # Ensure that the upload scheduler is running
    upload_ground_truth(datetime.utcnow() - timedelta(hours=1))

In [54]:
def stop():
    global stop_scheduler
    stop_scheduler = True  # Stop the upload scheduler

In [55]:
start_predictions()  # Start the prediction task every 20 seconds
start_upload()   # Start the upload task every minute

.....-...-..-...--..
Uploading 20 records to s3://sagemaker-us-east-1-084375567266/aai-540-group-3-final-project/ground_truth_data/2025-02-25/2025/02/25/04/0221.jsonl
-..--..---....---..-
-..--..----.-.-.-.-.
-..-.---.-...--..-.-
.--.--.--....--..-.-
.-..-..-.-..-.-----.
Uploading 100 records to s3://sagemaker-us-east-1-084375567266/aai-540-group-3-final-project/ground_truth_data/2025-02-25/2025/02/25/05/0321.jsonl
-.--...---.-..---.--
...-...-.-.-...-....
.-.-....--.-..-..--.
.......--..-..-.---.
--..-.------.----..-
....---.-..--...--..
Uploading 120 records to s3://sagemaker-us-east-1-084375567266/aai-540-group-3-final-project/ground_truth_data/2025-02-25/2025/02/25/05/0421.jsonl
----.--.--..--...--.
----...-.-.----..--.
..---..---..----.--.
--....-...-.--.-..-.
-...-----.--.--...-.
.-.-..---....--.----
Uploading 120 records to s3://sagemaker-us-east-1-084375567266/aai-540-group-3-final-project/ground_truth_data/2025-02-25/2025/02/25/05/0522.jsonl
..-..-.....---...-..
...-.----..-..

In [22]:
# To stop the scheduler at any point, set the flag to True
# Example: to stop the schedule, call the following line
stop()
# clear the stored Employee IDs after uploading
employee_ids_store.clear()

Upload scheduler stopped.
Upload scheduler stopped.


In [25]:
# View Capture data
gt_file_splits = S3Downloader.read_file("s3://sagemaker-us-east-1-084375567266/aai-540-group-3-final-project/ground_truth_data/2025-02-25/2025/02/25/04/1647.jsonl").split("\n")
for split in gt_file_splits:
    json.loads(split)
    display(split)

'{"groundTruthData": {"data": "0", "encoding": "CSV"}, "eventMetadata": {"eventId": "47280"}, "eventVersion": "0"}'

'{"groundTruthData": {"data": "0", "encoding": "CSV"}, "eventMetadata": {"eventId": "51909"}, "eventVersion": "0"}'

'{"groundTruthData": {"data": "0", "encoding": "CSV"}, "eventMetadata": {"eventId": "28878"}, "eventVersion": "0"}'

'{"groundTruthData": {"data": "0", "encoding": "CSV"}, "eventMetadata": {"eventId": "62022"}, "eventVersion": "0"}'

'{"groundTruthData": {"data": "1", "encoding": "CSV"}, "eventMetadata": {"eventId": "14988"}, "eventVersion": "0"}'

...

'{"groundTruthData": {"data": "1", "encoding": "CSV"}, "eventMetadata": {"eventId": "40736"}, "eventVersion": "0"}'

'{"groundTruthData": {"data": "1", "encoding": "CSV"}, "eventMetadata": {"eventId": "62913"}, "eventVersion": "0"}'

'{"groundTruthData": {"data": "0", "encoding": "CSV"}, "eventMetadata": {"eventId": "3475"}, "eventVersion": "0"}'

'{"groundTruthData": {"data": "0", "encoding": "CSV"}, "eventMetadata": {"eventId": "531"}, "eventVersion": "0"}'

'{"groundTruthData": {"data": "0", "encoding": "CSV"}, "eventMetadata": {"eventId": "72055"}, "eventVersion": "0"}'